# VELOX Processing Test Notebook

This notebook demonstrates loading a dataset, applying a quality mask, processing it with the pushbroom function, and visualizing the output.

Make sure your package is in the Python path or installed as a package so that you can import `concatenate_images3` from `velox_tools.processing`.

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

# Import the processing function from your package
from velox_tools.processing import concatenate_images3

# Optional: set matplotlib style
plt.style.use('seaborn-white')

## Load and Preprocess Dataset

Here we load a NetCDF dataset, sort it by time, and apply a quality mask based on the `BT_2D` variable. Only time steps where any data is present will be kept.

In [ ]:
# Load dataset and sort by time
ds = xr.open_dataset('../../Mueller_et_al_2024/data/predicted/circles_georeff/RF12_hl06.nc').sortby('time')

# Create a quality mask from the first band of BT_2D: True if any non-null value exists in x and y
quality_mask = ds['BT_2D'].isel(band=0).notnull().any(dim=['x', 'y'])

# Select only the time steps where the quality mask is True
ds = ds.sel(time=quality_mask)

print('Dataset dimensions after applying quality mask:')
print(ds)

## Processing with the Pushbroom Function

Next, we process the dataset using the `concatenate_images3` function with a slicing position of 100 and specifying mode as `'xarray'`.

In [ ]:
# Process the dataset using the pushbroom function
cc = concatenate_images3(ds, mode='xarray', slicing_position=100)

print('Processed dataset dimensions:')
print(cc)

## Interpolate Missing Values

Now we interpolate missing values along the time dimension using the nearest method.

In [ ]:
# Interpolate missing values along time
cc2 = cc.interpolate_na(dim='time', method='nearest')

print('Dataset after interpolation:')
print(cc2)

## Visualize the Result

Finally, we plot the first band of the processed `BT_2D` variable as an image. The `robust=True` flag is used for robust color scaling and the figure size is set to be wide and short.

In [ ]:
# Plot the first band of BT_2D using imshow
cc2.BT_2D.isel(band=0).plot.imshow(robust=True, figsize=(20, 1))
plt.title('Processed BT_2D (Band 0)')
plt.show()